<img src="../images/QISKit-c.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="250 px" align="left">

The latest version of this notebook is available [here.](https://github.com/QISKit/qiskit-tutorial) For more information about how to use the IBM Q experience (QX), see 
[this](https://quantumexperience.ng.bluemix.net) link.


## Running Quantum Examples Using IBM Data Science Experience

This example is copied from the original [getting_started.ipynb](https://github.com/QISKit/qiskit-tutorial/blob/master/1_introduction/getting_started.ipynb). It is tweaked (changes in the first code cell), so that it can be directly run using [IBM Data Science Experience](https://datascience.ibm.com) (IBM DSX). If you want to run other Quantum notebooks using IBM DSX, simply copy the first *code cell* and paste it at the beginning of your *other* notebook.

***

### About IBM Data Science Experience

IBM Data Science Experience (IBM DSX) is an interactive, collaborative, cloud-based environment where data scientists can use multiple tools to activate their insights.  

If you just starting with QISkit and looking for a webhosted Jupyter notebook environment, IBM DSX is an excellent option. It is fairly easy to get your quantum notebook running in the Jupyter notebook environment provided by IBM DSX. Moreover, it provides a platform where you can create your Quantum notebooks, invite fellow researchers to collaborate or simply share your work within the community.



### What you need to do

This example illustrates how to tweak your notebook so that it is ready to be run using IBM DSX. 

* In the first code cell, make sure to update the value of **`QX_API_token`** See the heading below that instructs how to get this information. 
* To run any other Quantum experience notebooks in IBM  DSX, just copy the first *code cell* of this notebook and paste it as the first *code cell* of your '*other*' notebook then just run it. 


### Getting your API token

The API access token allows you to connect to the IBM Quantum Experience API.  Here is how you can obtain this personal API access token.

1. If you do not have an account for [IBM Q experience](https://quantumexperience.ng.bluemix.net) yet, sign up for one by following the instructions . 
2. Login to [IBM Q experience](https://quantumexperience.ng.bluemix.net).
3. Click on your username that is displayed near the top right corner, and select *My Account*
4. The *My Account* page has a section called *Personal Access Token* . Just copy the token displayed there. 
5. Update the value of **`QX_API_TOKEN`** in the first code cell with this API token you just copied in previous step.
  

### Credits and Contributions
1. Ismael Faro, Jay Gambetta, Andrew Cross. See: [getting_started.ipynb](https://github.com/QISKit/qiskit-tutorial/blob/master/1_introduction/getting_started.ipynb).
2. See this [public gist](https://gist.github.com/dtmcclure/a1299fc99d12f5dc37ad6400fb8bbf2a) which has been used as a reference while preparing this notebook. (Github user [dtmcclure](https://github.com/dtmcclure) ) 
3. Ninad Sathaye (code changes for running the notebook on IBM Data Science Experience)

## The 'first code cell' follows (customization for IBM DSX):

In [ ]:
# This cell does some preparatory work to run this Jupyter notebook using 
# IBM Data Science Experience (datascience.ibm.com). All the other cells remain the same as in the original example. 

# --------------------------------------------------------------------------------------------------
# IMPORTANT NOTES: 
# 1) Log on to your QX account and get your API token. You can get it from the Accounts page 
# (the top-right icon on QX website) Copy-Paste the API token below.BE SURE TO ENCLOSE IN QUOTES ("")
# 2) If you are sharing this notebook, MAKE SURE TO REMOVE the API token string before sharing! 
#   Alternatively, you can modify the code below and separately create and save Qconfig.py file with 
#   API token.
# --------------------------------------------------------------------------------------------------
QX_API_TOKEN = "PUT_YOUR_API_TOKEN_HERE"

import sys
import os

try:
    assert(sys.version_info.major > 2)
except AssertionError:          
    print("This code requires Python 3.5 or beyond. Your version: {}.{}".format(sys.version_info.major,sys.version_info.minor))
    raise

# DO NOT CHANGE THE FOLLOWING assertion
try:
    assert(QX_API_TOKEN != "PUT_YOUR_API_TOKEN_HERE")
except AssertionError:
    print("Update the value of QX_API_TOKEN first!")
    raise
    
# importlib.reload is used to reload the Qconfig module. Fixes any update problems with Qconfig.APItoken 
import importlib

# We need visibility to Qconfig.py module add current dir your sys.path
sys.path.append("./")

# Install qiskit package
!pip install qiskit
!rm -rf Qconfig.py 
!touch Qconfig.py

apitoken_str = 'APItoken=\"{}\"'.format(QX_API_TOKEN)
config_str = "config = { \"url\": 'https://quantumexperience.ng.bluemix.net/api'}"

with open('Qconfig.py', 'w') as fil:
    fil.write(apitoken_str + "\n")
    fil.write(config_str + "\n")
    
# Now import all the modules from qiskit such as QuantumProgram, QuantumCircuit, QuantumRegister etc.
# Optionally, you can modify the qiskit import line below to do specific module imports. 
from qiskit import *
import Qconfig

try:
    # Reload Qconfig module again to make sure the APItoken is updated
    importlib.reload(Qconfig)
except NameError as e:
    print("Name error: ", e.args)
except:
    print("unable to reload the module, continuing the execution..")
    
print("Qconfig.APItoken = ", Qconfig.APItoken)

## Basic Concept

The basic concept of our quantum program is an array of quantum circuits. The program workflow consists of three stages: [Build](#sectionB), [Compile](#sectionC), and [Run](#sectionR). Build allows you to make different quantum circuits that represent the problem you are solving; Compile allows you to rewrite them to run on different backends (simulators/real chips of different [quantum volumes](http://ibm.biz/qiskit-quantum-volume), sizes, fidelity, etc); and Run launches the jobs.  After the jobs have been run, the data is collected. There are methods for putting this data together, depending on the program. This either gives you the answer you wanted, or allows you to make a better program for the next instance.

### Building your program: Create it  <a id='sectionB'></a>

First you need to import the QuantumProgram package from QISKit.

The basic elements needed for your first program are the QuantumProgram, a Circuit, a Quantum Register, and a Classical Register.

In [ ]:
# Creating Programs
# create your first QuantumProgram object instance.
qp = QuantumProgram()

# Creating Registers
# create your first Quantum Register called "qr" with 2 qubits 
qr = qp.create_quantum_register('qr', 2)
# create your first Classical Register  called "cr" with 2 bits
cr = qp.create_classical_register('cr', 2)

# Creating Circuits
# create your first Quantum Circuit called "qc" involving your Quantum Register "qr"
# and your Classical Register "cr"
qc = qp.create_circuit('qc', [qr], [cr])

Another option for creating your QuantumProgram instance is to define a dictionary with all the necessary components of your program.

In [ ]:
Q_SPECS = {
    'circuits': [{
        'name': 'Circuit',
        'quantum_registers': [{
            'name': 'qr',
            'size': 4
        }],
        'classical_registers': [{
            'name': 'cr',
            'size': 4
        }]}],
}

The required element for a Program is a "circuits" array. Within "circuits", the required field is "name"; it can have several Quantum Registers and Classical Registers. Every register must have a name and the number of each element (qubits or bits).

After that, you can use this dictionary definition as the specs of one QuantumProgram object to initialize it.

In [ ]:
qp = QuantumProgram(specs=Q_SPECS)

You can also get every component from your new qp to use.

In [ ]:
# Get the components.

# get the circuit by Name
circuit = qp.get_circuit('Circuit')

# get the Quantum Register by Name
quantum_r = qp.get_quantum_register('qr')

# get the Classical Register by Name
classical_r = qp.get_classical_register('cr')

### Building your program: Add Gates to your Circuit
After you create the circuit with its registers, you can add gates to manipulate the registers. Below is a list of the gates you can use in the QX.

You can find extensive information about these gates and how use them in our [Quantum Experience User Guide](https://quantumexperience.ng.bluemix.net/qstage/#/tutorial?sectionId=71972f437b08e12d1f465a8857f4514c&pageIndex=2). 

In [ ]:
# Pauli X gate to qubit 1 in the Quantum Register "qr" 
circuit.x(quantum_r[1])

# Pauli Y gate to qubit 2 in the Quantum Register "qr" 
circuit.y(quantum_r[2])

# Pauli Z gate to qubit 3 in the Quantum Register "qr" 
circuit.z(quantum_r[3])

# CNOT (Controlled-NOT) gate from qubit 0 to qubit 2
circuit.cx(quantum_r[0], quantum_r[2])

# add a barrier to your circuit
circuit.barrier()

# H (Hadamard) gate to qubit 0 in the Quantum Register "qr" 
circuit.h(quantum_r[0])

# S Phase gate to qubit 0
circuit.s(quantum_r[0])

# T Phase gate to qubit 1
circuit.t(quantum_r[1])

# identity gate to qubit 1
circuit.iden(quantum_r[1])

# first physical gate: u1(lambda) to qubit 0
circuit.u1(0.3, quantum_r[0])

# second physical gate: u2(phi,lambda) to qubit 1
circuit.u2(0.3, 0.2, quantum_r[1])

# second physical gate: u3(theta,phi,lambda) to qubit 2
circuit.u3(0.3, 0.2, 0.1, quantum_r[2])

# rotation around the x-axis to qubit 0
circuit.rx(0.2, quantum_r[0])

# rotation around the y-axis to qubit 1
circuit.ry(0.2, quantum_r[1])

# rotation around the z-axis to qubit 2
circuit.rz(0.2, quantum_r[2])

# Classical if, from qubit2 gate Z to classical bit 1
# circuit.z(quantum_r[2]).c_if(classical_r, 0)

# measure gate from qubit 0 to classical bit 0
circuit.measure(quantum_r[0], classical_r[0])
circuit.measure(quantum_r[1], classical_r[1])
circuit.measure(quantum_r[2], classical_r[2])

If you want to see what circuits are in your program you can use the get_circuit_names()

In [ ]:
qp.get_circuit_names()

### Extract QASM

You can obtain a QASM representation of your code.

In [ ]:
# QASM from a program

QASM_source = qp.get_qasm('Circuit')

print(QASM_source)

### Compile and Run or Execute <a id='sectionC'></a>

First we need to choose the backend. Lets start with the local simulator 

In [ ]:
backend = 'local_qasm_simulator' 
circuits = ['Circuit']  # Group of circuits to execute

Next we need to compile the circuits into a quantum object which we call qobj

In [ ]:
qobj=qp.compile(circuits, backend) # Compile your program

Then you can run your program. Using wait and timeout we can check the execution result every 2 seconds and timeout if the job is not run in 240 seconds. 

In [ ]:
result = qp.run(qobj, wait=2, timeout=240)
print(result)

When you run a program, the result will be a new object that contains the data, status, and a copy of the qobj. The status of the results can be obtian using print and will one of the follow:

```
JOB_STATUS = {
    inProgress: 'RUNNING',
    errorOnCreate: 'ERROR_CREATING_JOB',
    errorExecuting: 'ERROR_RUNNING_JOB',
    completed: 'COMPLETED'
  };
``` 

### Result
You can access the result via the function 

```get_counts('name')```. 

In [ ]:
result.get_counts('Circuit')

In addition to getting the number of times each output was seen, you can get the circuit which was run in QASM format.
For this simulation, the compiled circuit is not much different from the input circuit. Each single-qubit gate has been expressed as a u1, u2, or u3 gate.

In [ ]:
ran_qasm = result.get_ran_qasm('Circuit')

print(ran_qasm)

You can use 

```qp.execute(circuits)```

to combine the compile and run in a single step.

In [ ]:
out = qp.execute(circuits, backend, wait=2, timeout=240)
print(out)

#### Compile Parameters
```qp.compile(circuits, backend='ibmqx2', shots=1024, max_credits=3, basis_gates=None, coupling_map=None, seed=None)```

     - circuits: Array of circuits to compile
     - backend: Backend 
        ['ibmqx_qasm_simulator',    # qasm simulator 
         'ibmqx2',                  # online real chip with 5 qubits
         'ibmqx3',                  # online real chip with 16 qubits
         'local_unitary_simulator', # local unitary simulator 
         'local_qasm_simulator']    # local simulator 
     - config: configure options for the compiler 
     - shots: Number of shots, only for real chips and qasm simulators
     - max_credits: Maximum number of credits to spend in the executions. If the executions cost more than your available credits, the job is aborted
     - basis_gates: the base gates by default are: u1, u2, u3, cx, id
     - coupling_map: Object that represents the physical/topological layout of a chip.
     - seed: For the qasm simulator if you want to set the initial seed. 
#### Run Parameters
```qp.run(qobj, wait=5, timeout=60)```
     - the qobj to be run.
     - wait: Time to wait before checking if the execution is COMPLETED.
     - timeout: Timeout of the execution.
#### Execute Parameters 
Execute has the combined parameters of compile and run.

```qp.execute(circuits, backend, shots=1024, max_credits=3, basis_gates=None, wait=5, timeout=60, basis_gates=None, coupling_map=None)```

### Execute on a Real Device<a id='sectionR'></a>

In [ ]:
backend = 'ibmqx2'   # Backend where you execute your program; in this case, on the Real Quantum Chip online 
circuits = ['Circuit']   # Group of circuits to execute
shots = 1024           # Number of shots to run the program (experiment); maximum is 8192 shots.
max_credits = 3          # Maximum number of credits to spend on executions. 
qp.set_api(Qconfig.APItoken, Qconfig.config['url']) # set the APIToken and API url

result_real = qp.execute(circuits, backend, shots=shots, max_credits=3, wait=10, timeout=240)

Like before the counts for both executions can be obtained using ```get_counts('name')``` 

In [1]:
%run "../version.ipynb"

Package,Version
QISKit,0.4.8
IBMQuantumExperience,>= 1.8.26
numpy,">= 1.13, < 1.14"
scipy,">= 0.19, < 0.20"
matplotlib,">= 2.0, < 2.1"


In [ ]:
result_real.get_counts('Circuit')

In [ ]:
result_real2.get_counts('Circuit')